In [1]:
import numpy as np
from osgeo import ogr, gdal
import rasterio
import json

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path
import re

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

In [3]:
area_cover = '/home/shrayank_mistry/Modules/project-mum-pune/mask.tif'
area = rasterio.open(area_cover, count = 1)

area = np.array(area.read(1))

In [4]:
area_dem = '/home/shrayank_mistry/Modules/project-mum-pune/dem_clipped.tif'
dem = rasterio.open(area_dem, count = 1)

dem = np.array(dem.read(1))
print(dem.shape)

(2721, 3610)


In [5]:
import rasterio
from rasterio.enums import Resampling

# whole-numbers indicate upscaling, fractions indicate downscaling
upscale_factor = 3


with rasterio.open('/home/shrayank_mistry/Modules/project-mum-pune/dem_clipped.tif') as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

In [6]:
area = np.swapaxes(area, 1, 0)
print(area.shape)

up_dem = data[0]
up_dem.shape

(10500, 8273)


(8163, 10830)

In [7]:
dem_data, mask_data = up_dem, area
dem_data = dem_data.T


print(dem_data.shape, mask_data.shape)
width, height = 10500, 8163

dem_data, mask_data = dem_data[:width,:height], mask_data[:width, :height]
print(dem_data.shape, mask_data.shape)

(10830, 8163) (10500, 8273)
(10500, 8163) (10500, 8163)


In [8]:
ahp_map = {

'urban': 0.29,
'farms': 0.239,
'dense-forest': 0.207,
'water': 0.13,
'fallow': 0.067,
'sparse-forest': 0.049,
'barren-land': 0.019,
'unclassified':7,

}

class_map = {

    0: 'unclassified',
    1: 'water',
    2: 'dense-forest',
    3: 'sparse-forest',
    4: 'barren-land',
    5: 'urban',
    6: 'farms',
    7: 'fallow',

}


In [9]:
from copy import deepcopy
mask_copy = deepcopy(mask_data)

In [10]:
def set_weights(c):
    c_str = class_map.get(c)
    return ahp_map[c_str]

In [11]:
set_weights_vtr = np.vectorize(set_weights)
mask_copy = set_weights_vtr(mask_copy)

In [12]:
np.unique(mask_copy, return_counts = True)

(array([0.019, 0.049, 0.067, 0.13 , 0.207, 0.239, 0.29 , 7.   ]),
 array([19822048, 17097029, 28893954,  7807154,  3858436,  5940554,
         2291737,      588]))

In [13]:
np.unique(mask_data, return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int16),
 array([     588,  7807154,  3858436, 17097029, 19822048,  2291737,
         5940554, 28893954]))

In [14]:
from osgeo import ogr
import json

file = ogr.Open("/home/shrayank_mistry/Modules/project-mum-pune/route-files/source-point.shp")
source_shp = file.GetLayer(0)

feature = source_shp.GetFeature(0)
source_shp = feature.ExportToJson()

# source_shp
source_shp = json.loads(source_shp)
start_ext = source_shp['geometry']['coordinates']

In [15]:
file = ogr.Open("/home/shrayank_mistry/Modules/project-mum-pune/route-files/destination-point.shp")
destination_shp = file.GetLayer(0)

feature = destination_shp.GetFeature(0)
destination_shp = feature.ExportToJson()

destination_shp

# source_shp
destination_shp = json.loads(destination_shp)
end_ext = destination_shp['geometry']['coordinates']

In [16]:
print(start_ext, end_ext)

[300378.7763074944, 2104126.3099621385] [366123.3883685307, 2063699.8154670105]


In [17]:
from osgeo import ogr, gdal
import os
import subprocess
import rasterio

In [18]:
path = '/home/shrayank_mistry/Modules/project-mum-pune/raster.tif' 

data = rasterio.open(path)
print(data.bounds)

extent = data.bounds

left, bottom, right, top = extent[0], extent[1], extent[2], extent[3]
print(left, bottom, right, top)

BoundingBox(left=282680.0, bottom=2035820.0, right=387680.0, top=2118550.0)
282680.0 2035820.0 387680.0 2118550.0


In [19]:
width = round(right - left)
height = round(top - bottom)

print("Width and Height of Raster")
print(width, height)

Width and Height of Raster
105000 82730


In [20]:
# width = round(right - left)
# height = round(top - bottom)

# print(width, height)

# start_pixel, end_pixel = [0, 0], [0, 0]
# start_flag, end_flag = True, True

# for i in range(height):
#     for j in range(width):

#         if (not start_flag) and (not end_flag):
#             break
        
#         # print(round(left + j), round(top - i))
#         if (start_flag and ((round(left + j)) == round(start_ext[0])) and ((round(top - i)) == round(start_ext[1]))):
#             start_pixel = [i, j]
#             start_flag = False

        
#         if (end_flag and ((round(left + j)) == round(end_ext[0])) and ((round(top - i)) == round(end_ext[1]))):
#             end_pixel = [i, j]
#             end_flag = False

In [21]:
start_pixel, end_pixel = [14424, 17699], [54850, 83443]

In [22]:
start_pixel = np.array(start_pixel)
start_pixel = start_pixel / 10

start_pixel = list(np.rint(start_pixel))
start_pixel = list(np.array(start_pixel, dtype = 'int'))

end_pixel = np.array(end_pixel)
end_pixel = end_pixel / 10

end_pixel = list(np.rint(end_pixel))
end_pixel = list(np.array(end_pixel, dtype = 'int'))

### Dijkstra's Algorithm for Anistropic Accumulated-Cost Calculation

In [23]:
def condition_check(start, end):
    if (start[0] == end[0]) and (start[1] == end[1]):
        return False
    return True

In [24]:
import math
from queue import PriorityQueue

In [25]:
from copy import deepcopy
mask_copy_t = deepcopy(mask_copy)

In [26]:
mask_copy = mask_copy * 5000

In [27]:
# mask_copy = mask_copy.T
# print(mask_copy.shape)

# dem_data = dem_data.T
# print(dem_data.shape)

In [28]:
np.unique(mask_copy, return_counts = True)

(array([   95.,   245.,   335.,   650.,  1035.,  1195.,  1450., 35000.]),
 array([19822048, 17097029, 28893954,  7807154,  3858436,  5940554,
         2291737,      588]))

In [29]:
def c_anist_cost(i, j, x, y, mu = 10, wt = 2):

    mu_sqr = mu * mu
    h_diff = dem_data[i][j] - dem_data[x][y]
    h_sqr = h_diff * h_diff
    c_dv = (mask_copy[i][j] + mask_copy[x][y]) / 2
    cst = np.sqrt(mu_sqr + h_sqr) * (c_dv + math.atan(h_diff / mu) * wt) + acc_cost[i][j]

    # print(cst)
    return cst


In [30]:
def get_neigh_cost(i, j):
    arr = []
    #(1) col - 1, row - 1
    if (j - 1 >= 0) and (i - 1 >= 0):
        # arr.append([c_anist_cost(i, j, i - 1, j - 1), [i - 1, j - 1], [i, j]])
        # acc_cost[i - 1][j - 1] = min(c_anist_cost(i, j, i - 1, j - 1), acc_cost[i - 1][j - 1])

        if (acc_cost[i - 1][j - 1] > c_anist_cost(i, j, i - 1, j - 1)):
            parent[i - 1][j - 1] = i, j
            acc_cost[i - 1][j - 1] = c_anist_cost(i, j, i - 1, j - 1)
        arr.append([acc_cost[i - 1][j - 1], [i - 1, j - 1], [i, j]])
    else:
        arr.append(math.inf)

    #(2) col, row - 1
    if (i - 1 >= 0):
        # arr.append([c_anist_cost(i, j, i - 1, j), [i - 1, j], [i, j]])
        # acc_cost[i - 1][j] = min(c_anist_cost(i, j, i - 1, j), acc_cost[i - 1][j])

        if (acc_cost[i - 1][j] > c_anist_cost(i, j, i - 1, j)):
            parent[i - 1][j] = i, j
            acc_cost[i - 1][j] = c_anist_cost(i, j, i - 1, j)
        arr.append([acc_cost[i - 1][j], [i - 1, j], [i, j]])
    else:
        arr.append(math.inf)

    #(3) col + 1, row - 1
    if (j + 1 < 8163) and (i - 1 >= 0):
        # arr.append([c_anist_cost(i, j, i - 1, j + 1), [i - 1, j + 1], [i, j]])
        # acc_cost[i - 1][j + 1] = min(c_anist_cost(i, j, i - 1, j + 1), acc_cost[i - 1][j + 1])

        if (acc_cost[i - 1][j + 1] > c_anist_cost(i, j, i - 1, j + 1)):
            parent[i - 1][j + 1] = i, j
            acc_cost[i - 1][j + 1] = c_anist_cost(i, j, i - 1, j + 1)
        arr.append([acc_cost[i - 1][j + 1], [i - 1, j + 1], [i, j]])
    else:
        arr.append(math.inf)

    #(4) col - 1, row
    if (j - 1 >= 0):
        # arr.append([c_anist_cost(i, j, i, j - 1), [i, j - 1], [i, j]])
        # acc_cost[i][j - 1] = min(c_anist_cost(i, j, i, j - 1), acc_cost[i][j - 1])

        if (acc_cost[i][j - 1] > c_anist_cost(i, j, i, j - 1)):
            parent[i][j - 1] = i, j
            acc_cost[i][j - 1] = c_anist_cost(i, j, i, j - 1)
        arr.append([acc_cost[i][j - 1], [i, j - 1], [i, j]])
    else:
        arr.append(math.inf)

    #(5) col + 1, row
    if (j + 1 < 8163):
        # arr.append([c_anist_cost(i, j, i, j + 1), [i, j + 1], [i, j]])
        # acc_cost[i][j + 1] = min(c_anist_cost(i, j, i, j + 1), acc_cost[i][j + 1])

        if (acc_cost[i][j + 1] > c_anist_cost(i, j, i, j + 1)):
            parent[i][j + 1] = i, j
            acc_cost[i][j + 1] = c_anist_cost(i, j, i, j + 1)
        arr.append([acc_cost[i][j + 1], [i, j + 1], [i, j]])
    else:
        arr.append(math.inf)

    #(6) col - 1, row + 1
    if (j - 1 >= 0) and (i + 1 < 10500):
        # arr.append([c_anist_cost(i, j, i + 1, j - 1), [i + 1, j - 1], [i, j]])
        # acc_cost[i + 1][j - 1] = min(c_anist_cost(i, j, i + 1, j - 1), acc_cost[i + 1][j - 1])

        if (acc_cost[i + 1][j - 1] > c_anist_cost(i, j, i + 1, j - 1)):
            parent[i + 1][j - 1] = i, j
            acc_cost[i + 1][j - 1] = c_anist_cost(i, j, i + 1, j - 1)
        arr.append([acc_cost[i + 1][j - 1], [i + 1, j - 1], [i, j]])
    else:
        arr.append(math.inf)

    #(7) col, row + 1
    if (i + 1 < 10500):
        # arr.append([c_anist_cost(i, j, i + 1, j), [i + 1, j], [i, j]])
        # acc_cost[i + 1][j] = min(c_anist_cost(i, j, i + 1, j), acc_cost[i + 1][j])

        if (acc_cost[i + 1][j] > c_anist_cost(i, j, i + 1, j)):
            parent[i + 1][j] = i, j
            acc_cost[i + 1][j] = c_anist_cost(i, j, i + 1, j)
        arr.append([acc_cost[i + 1][j], [i + 1, j], [i, j]])
    else:
        arr.append(math.inf)

    #(8) col + 1, row + 1
    if (j + 1 < 8163) and (i + 1 < 10500):
        # arr.append([c_anist_cost(i, j, i + 1, j + 1), [i + 1, j + 1], [i, j]])
        # acc_cost[i + 1][j + 1] = min(c_anist_cost(i, j, i + 1, j + 1), acc_cost[i + 1][j + 1])

        if (acc_cost[i + 1][j + 1] > c_anist_cost(i, j, i + 1, j + 1)):
            parent[i + 1][j + 1] = i, j
            acc_cost[i + 1][j + 1] = c_anist_cost(i, j, i + 1, j + 1)
        arr.append([acc_cost[i + 1][j + 1], [i + 1, j + 1], [i, j]])
    else:
        arr.append(math.inf)
    

    return arr

In [31]:
start_pixel, end_pixel = [1770, 1442], [8344, 5485]
print(start_pixel, end_pixel)

[1770, 1442] [8344, 5485]


In [32]:
dim = mask_copy.shape
print(dim)

(10500, 8163)


In [33]:
print(mask_copy.shape, dem_data.shape)

(10500, 8163) (10500, 8163)


In [34]:
visited = np.zeros((dim))
acc_cost = np.full((dim), math.inf)

print(visited.shape, acc_cost.shape)

(10500, 8163) (10500, 8163)


In [35]:
print(start_pixel, end_pixel)

[1770, 1442] [8344, 5485]


In [37]:
Q = PriorityQueue()
s_pixel, e_pixel = start_pixel, end_pixel

# mask_copy.shape, dem_data.shape

visited = np.zeros((dim))
acc_cost = np.full((dim), math.inf)

acc_cost[s_pixel[0]][s_pixel[1]] = 0
visited[s_pixel[0]][s_pixel[1]] = 1

parent = np.full((dim), None)
parent[s_pixel[0]][s_pixel[1]] = -1, -1


path = list()

while (condition_check(s_pixel, e_pixel)):
    # print('in')
    i, j = s_pixel

    neighbours_cost = get_neigh_cost(i, j)
    for nc in neighbours_cost:
        if nc == math.inf:
            continue
        else:
            Q.put(nc)
    
    # check if current-best is visited OR not
    bst = Q.get()
    m, n = bst[1][0], bst[1][1]

    while True:
        if visited[m][n] == 0:
            break
        bst = Q.get()
        m, n = bst[1][0], bst[1][1]

    # set-visited
    visited[m][n] = 1
    path.append([m, n])
    # print(m, n)

    parent[m][n] = bst[2][0], bst[2][1]

    s_pixel = [m, n]

In [38]:
print(acc_cost[end_pixel[0]][end_pixel[1]])

10502171.269605746


In [39]:
!rm -rf path.txt

In [40]:
path = []
pr = parent[end_pixel[0]][end_pixel[1]]

# cnt = 15000
while (pr[0] != -1) and (pr[1] != -1):
    path.append(pr)
    # path.append('-')
    with open('/home/shrayank_mistry/Modules/project-mum-pune/routes-shape/dijk-route/path.txt', 'a') as f:
        f.write(str(pr) + '\n')

    # cnt = cnt - 1
    # if cnt == 0:
    #     break

    pr = parent[pr[0]][pr[1]]

In [41]:
path_list = []
with open('/home/shrayank_mistry/Modules/project-mum-pune/routes-shape/dijk-route/path.txt', 'r') as f:
    for point in f:
        path_list.append(point)

In [42]:
len(path_list)

7646

In [43]:
point = path_list[0].replace('\n', '').split(' ')
pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])
print(pi, pj)

8343 5484


In [44]:
path_list.reverse()

In [45]:
%%time
ordinates_dict = {}


for i in range(len(path_list)):
    point = path_list[i].replace('\n', '').split(' ')
    pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

    ext_i, ext_j = left + (pi * 10), top - (pj * 10)

    # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
    ordinates_dict[pi, pj] = [ext_i, ext_j]

CPU times: user 30.7 ms, sys: 12 ms, total: 42.7 ms
Wall time: 112 ms


In [46]:
ordinates_dict
ordinates_list = []
for key, value in ordinates_dict.items():
    ordinates_list.append(value)

In [47]:
import shapefile
w = shapefile.Writer('/home/shrayank_mistry/Modules/project-mum-pune/routes-shape/dijk-route/shapefiles/test/multipoint')
w.field('name', 'C')

w.multipoint(ordinates_list) 
w.record('multipoint1')

w.close()

In [48]:
road_lenght = 0
index = 1
for _ in ordinates_list[1:]:
    i, j = ordinates_list[index][0], ordinates_list[index][1]
    x, y = ordinates_list[index - 1][0], ordinates_list[index - 1][1]

    if (x - i == 10.0) and (y - j == -10.0):
        road_lenght += math.sqrt(2 * 100)
    else:
        road_lenght += 10 

In [49]:
print(f'Current Road-length = {road_lenght / 1000} kms')

Current Road-length = 76.45 kms
